In [71]:
import numpy as np
import pandas as pd

### 1) une colonne motVedette (transformation/simplification de la prononciation d'origine) en utilisant le mapping du fichier Buchanan_PronChar_counts.csv

#### Import des données

In [72]:
# préciser le chemin du fichier à lire

cheminDuFichierDictionnaire = "./../data/dictionary/1757_Buchanan-J.csv"
cheminDuFichierMapping = "./../data/dictionary/Buchanan_PronChar_counts.csv" 

# demander à pandas de lire le fichier et garder ce tableau accessible dans une variable
dfDicoBuch = pd.read_csv(cheminDuFichierDictionnaire, delimiter=";", encoding="utf8")
dfDicoMapping = pd.read_csv(cheminDuFichierMapping, delimiter=";", encoding="utf8")

In [73]:
# afficher à l'écran un échantillon du tableau
dfDicoBuch.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId
9203,2,оaʹk_e_n,NaN,NaN,NaN,a.,5.0,149
11932,5,săbbătāʹrĭans,NaN,NaN,NaN,n.,3.0,187
6912,5,indĭʹſsŏlūble,L.,NaN,NaN,a.,33.0,117
6513,4,illāʹbŏrāte,L.,NaN,NaN,a.,68.0,111
831,2,ăſſuāʹge,NaN,NaN,NaN,v.,5.0,31


In [74]:
dfDicoMapping.sample(n=5)

,char,occ,graph
13,ā,2079,a
16,ŏ,2797,o
19,h,2383,h
38,k,717,k
1,ʹ,12725,NaN


#### Aucun caractère n'a plus d'une correspondance dans la table de mapping : 

In [75]:
(dfDicoMapping['char'].value_counts()>1).sum()

0

#### Transformation de la prononciation en mot

In [76]:
# suppression des lignes avec un caractère n'ayant pas de correspondance
dfMapping = dfDicoMapping[~dfDicoMapping['graph'].isna()].copy()

In [77]:
def pronToMot(strBuch, dfMapping):
    res = ""
    
    for charBuch in strBuch:
        dfRowFinded = dfMapping[dfMapping['char']==charBuch]

        if len(dfRowFinded==1):
            res+=str(dfRowFinded.iloc[0]['graph'])
        else:
            res+=''

    return res

In [78]:
## test
pronToMot('hauʹ_gh_ty', dfMapping)

'haughty'

In [79]:
dfDicoBuchMot = dfDicoBuch.sample(n=1000).copy()

dfDicoBuchMot['motVedette'] = dfDicoBuchMot.apply(lambda x: pronToMot(x['pron'], dfMapping),1)

In [82]:
dfDicoBuchMot

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette
7146,5,inſtăbĭʹlĭty,L.,NaN,NaN,n.,52.0,120,instability
13240,4,sŭblieutĕʹnănt,NaN,NaN,NaN,n.,57.0,205,sublieutenant
13536,2,taʹbles,NaN,NaN,NaN,n.,8.0,209,tables
6722,4,impūʹtătive,NaN,NaN,NaN,a.,61.0,114,imputative
8516,4,mĕtŏʹny̆my,NaN,NaN,NaN,n.,65.0,139,metonyomy
...,...,...,...,...,...,...,...,...,...
7162,2,inſtrŭʹct,L.,NaN,NaN,v.,68.0,120,instruct
7253,3,intĕʹrprĕt,L.,NaN,NaN,v.,15.0,122,interpret
5338,1,frĭth,NaN,NaN,NaN,n.,69.0,93,frith
9665,2,păʹmphlet,NaN,NaN,NaN,n.,57.0,155,pamphlet


In [83]:
cheminDicoBuchMot = "./../data/dictionary/DicoMotVedette.csv"
dfDicoBuchMot.to_csv(cheminDicoBuchMot, sep=";", encoding="utf8", index=False )

### 2) une colonne Sampa et une colonne Ipa à partir des mots vedettes obtenus (transformation/croisement de données) en utilisant ./../data/dataframe/BigiRichSample.csv

#### 2) i. isoler les mots de Buchanan qui n'existent pas dans Bigi. Tenter de comprendre les différentes causes et tenter d'expliquer comment résoudre les différents problèmes (bonus)

In [67]:
cheminDuDicoBigi = "./../data/dictionary/2015_Bigi-B.dict" 

# ouvrir une matric exotique
#dfDicoBigi = pd.read_csv(cheminDuDicoBigi, delimiter=" [] ", encoding="utf8")

dfDicoBigi = pd.read_csv(cheminDuDicoBigi, delimiter=" \[\] ", encoding="utf8", engine='python', names=['hwd','pronSampa'], skiprows=8 )

# afficher à l'écran un échantillon du tableau
dfDicoBigi.head(n=5)

,hwd,pronSampa
0,a,@
1,a(2),eI
2,a'ready,@ r\ E d i:
3,a's,eI z
4,a.,eI
